In [1]:
"""
Sarsa is a online updating method for Reinforcement learning.

Unlike Q learning which is a offline updating method, Sarsa is updating while in the current trajectory.

You will see the sarsa is more coward when punishment is close because it cares about all behaviours,
while q learning is more brave because it only cares about maximum behaviour.
"""

from maze_env import Maze
from RL_brain import SarsaLambdaTable
import time

In [2]:
#conda list
import pandas as pd
import numpy as np
import time

Prepare a log.txt if not exists

https://www.freecodecamp.org/news/file-handling-in-python/#:~:text=In%20Python%2C%20you%20use%20the,it%20will%20return%20an%20error.

In [3]:
#
f = open("log.txt","a")

⚠️Sarsa(lambda)有两种更新Eligibility trace的方式

method1:self.eligibility_trace.loc[s, a] += 1
    
method2:self.eligibility_trace.loc[s, :] *= 0
        self.eligibility_trace.loc[s, a] = 1
        
代码中默认用的method2

In [4]:
import os.path
#https://stackoverflow.com/questions/82831/how-do-i-check-whether-a-file-exists-without-exceptions
if os.path.isfile('q_table_maze.csv'):
    print('Q-Table CSV Exists.')
    time.sleep(1)
    mode=input('1 for loading trained Q-table,2 for starting afresh. Please choose:')
    
    if mode=="1":
        #当后续的训练想要使用之前得到的Q-table时，导出后，传参给RL_brain.py中的__init__ function，最后赋值给self.q_table
        df=pd.read_csv('q_table_maze.csv',index_col=0)
        #df=pd.read_csv('q_table_output.csv') 默认读取时会自动设置新的index，但是这里我们需要的是将第一列(state)设置为index
        #df.columns发现输出时columns的dtype是int，而现在导入变成了object类型
        df.columns=[0,1,2,3]
    elif mode=="2":
        df=pd.DataFrame(columns=[0,1,2,3],dtype=np.float64) #initialize table
        f.write("\n\n-----Start Afresh-----\n")
else:
    df=pd.DataFrame(columns=[0,1,2,3],dtype=np.float64) #initialize table
    f.write("\n\n-----Start Afresh-----\n")

Q-Table CSV Exists.


In [5]:
# have a look at current Q-table
df

,0,1,2,3
"[5.0, 5.0, 35.0, 35.0]",-2.361697e-03,0.019841,-0.006954,-0.002486
"[5.0, 45.0, 35.0, 75.0]",-3.788782e-03,0.033267,-0.020790,-0.003304
"[45.0, 5.0, 75.0, 35.0]",-4.346641e-03,-0.012319,-0.016910,-0.002226
"[45.0, 45.0, 75.0, 75.0]",-6.753099e-04,-0.029701,-0.019900,-0.002288
terminal,0.000000e+00,0.000000,0.000000,0.000000
"[85.0, 5.0, 115.0, 35.0]",-5.634642e-04,-0.039404,-0.004326,-0.000452
"[5.0, 85.0, 35.0, 115.0]",-1.607414e-04,0.059397,-0.029701,-0.000139
"[5.0, 125.0, 35.0, 155.0]",-6.561000e-03,0.000888,0.083541,-0.000013
"[125.0, 5.0, 155.0, 35.0]",-3.225436e-08,-0.016128,-0.000073,-0.000015
"[125.0, 45.0, 155.0, 75.0]",0.000000e+00,0.000000,0.000000,-0.019900


In [6]:
epoches=10
def update():
    count_success=0
    count_steps=0
    start=time.time()
    for episode in range(epoches):
        # initial observation
        observation = env.reset()

        # RL choose action based on observation
        action = RL.choose_action(str(observation))

        # initial all zero eligibility trace
        RL.eligibility_trace *= 0
        
        #step counter
        steps=0

        while True:
            # fresh env
            env.render()

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            # RL choose action based on next observation
            action_ = RL.choose_action(str(observation_))

            # RL learn from this transition (s, a, r, s, a) ==> Sarsa
            RL.learn(str(observation), action, reward, str(observation_), action_)

            # swap observation and action
            observation = observation_
            action = action_
            
            steps+=1

            # break while loop when end of this episode
            if done:
                end=time.time()
                if reward==1:
                    count_success+=1
                count_steps+=steps
                print('Episode '+str(episode)+', total steps: '+str(steps)+', reward: ', end='')
                print('1') if reward==1 else print ('-1')
                break

    # 大循环完毕
    print('game over, AvgSteps= '+str(round(count_steps/epoches))+', success rate= '+str("%.2f%%"%(count_success/epoches*100)))
    # write into log.txt (append)
    f.writelines([time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(start)),"UTC——",
                  time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(end)),"UTC\n","Epoches: ",str(epoches)
                  ,", AvgSteps: ",str(round(count_steps/epoches)),", success rate: ",str("%.2f%%"%(count_success/epoches*100)),"\n"])
    f.close()
    env.destroy()

if __name__ == "__main__":
    env = Maze()
    #RL = SarsaLambdaTable(actions=list(range(env.n_actions)))
    RL = SarsaLambdaTable(actions=list(range(env.n_actions)),trained_q_table=df)

    env.after(epoches, update)
    env.mainloop()
    print('\r\nQ-table:\n')
    print(RL.q_table)

Episode 0, total steps: 6, reward: 1
Episode 1, total steps: 7, reward: 1
Episode 2, total steps: 8, reward: 1
Episode 3, total steps: 7, reward: 1
Episode 4, total steps: 6, reward: 1
Episode 5, total steps: 6, reward: 1
Episode 6, total steps: 6, reward: 1
Episode 7, total steps: 6, reward: -1
Episode 8, total steps: 6, reward: 1
Episode 9, total steps: 5, reward: -1
game over, AvgSteps= 6, success rate= 80.00%

Q-table:

                                         0         1         2         3
[5.0, 5.0, 35.0, 35.0]       -2.361697e-03  0.037878 -0.004407 -0.002486
[5.0, 45.0, 35.0, 75.0]      -3.788782e-03  0.053232 -0.020790 -0.007387
[45.0, 5.0, 75.0, 35.0]      -4.346641e-03 -0.012319 -0.016910  0.000858
[45.0, 45.0, 75.0, 75.0]     -6.753099e-04 -0.029701 -0.019900 -0.002288
terminal                      0.000000e+00  0.000000  0.000000  0.000000
[85.0, 5.0, 115.0, 35.0]     -5.634642e-04 -0.039404 -0.004326 -0.000452
[5.0, 85.0, 35.0, 115.0]     -1.607414e-04  0.081465 -0.02970

In [7]:
df=RL.q_table

In [8]:
df

,0,1,2,3
"[5.0, 5.0, 35.0, 35.0]",-2.361697e-03,0.037878,-0.004407,-0.002486
"[5.0, 45.0, 35.0, 75.0]",-3.788782e-03,0.053232,-0.020790,-0.007387
"[45.0, 5.0, 75.0, 35.0]",-4.346641e-03,-0.012319,-0.016910,0.000858
"[45.0, 45.0, 75.0, 75.0]",-6.753099e-04,-0.029701,-0.019900,-0.002288
terminal,0.000000e+00,0.000000,0.000000,0.000000
"[85.0, 5.0, 115.0, 35.0]",-5.634642e-04,-0.039404,-0.004326,-0.000452
"[5.0, 85.0, 35.0, 115.0]",-1.607414e-04,0.081465,-0.029701,-0.000139
"[5.0, 125.0, 35.0, 155.0]",-6.561000e-03,0.006448,0.109885,-0.000013
"[125.0, 5.0, 155.0, 35.0]",-3.225436e-08,-0.016128,-0.000073,-0.000015
"[125.0, 45.0, 155.0, 75.0]",0.000000e+00,0.000000,0.000000,-0.019900


In [9]:
# saving the dataframe
df.to_csv('q_table_maze.csv')